# DBSCAN Clustering - Temporal Pipeline

Full temporal pipeline matching the OPTICS approach: 13-feature multi-timeframe engineering,
adaptive eps selection via k-distance heuristic at each timestamp, pair stability,
formation/dissolution detection, permutation test, and artifact saving.

In [1]:
# IMPORTS
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
import warnings
import itertools
warnings.filterwarnings("ignore")

# Data Fetch

In [2]:
stocks = [
    "^GSPC",
    "NVDA", "TSM", "AVGO", "AMD", "INTC", "MU", "TXN", "QCOM", "ADI", "MCHP",
    "ASML", "AMAT", "LRCX", "KLAC", "TER", "ENTG", "NVMI", "TOELY",
    "ON", "NXPI", "STM", "LSCC", "MPWR", "QRVO", "SWKS", "ALAB", "CRDO",
    "ARM", "SNPS", "CDNS", "CEVA",
    "WDC", "STX",
    "GFS", "MRVL", "MTSI", "POWI", "SMTC", "VICR", "CAMT"
]

def fetch_data(stocks):
    data = yf.download(tickers=stocks, period="252d", interval="1h",
                       group_by='ticker', auto_adjust=True, threads=True)
    price_series_list = []
    for s in stocks:
        try:
            if s in data:
                series = data[s]['Close']
                series.name = s
                price_series_list.append(series)
        except Exception:
            pass
    if price_series_list:
        df = pd.concat(price_series_list, axis=1)
        df = df.groupby(df.index.date).apply(lambda g: g.ffill()).droplevel(0)
        return df
    return pd.DataFrame()

df = fetch_data(stocks)

[                       0%                       ]

[**                     5%                       ]  2 of 41 completed

[**                     5%                       ]  2 of 41 completed

[*****                 10%                       ]  4 of 41 completed

[******                12%                       ]  5 of 41 completed

...

[**********************78%************           ]  32 of 41 completed

[**********************80%*************          ]  33 of 41 completed

[**********************83%***************        ]  34 of 41 completed

[**********************88%*****************      ]  36 of 41 completed

[*********************100%***********************]  41 of 41 completed

## Feature Engineering (Multi-Timeframe)

In [3]:
if isinstance(df.columns, pd.MultiIndex):
    if 'Close' in df.columns.get_level_values(0):
        df = df['Close']
    elif 'Close' in df.columns.get_level_values(1):
        df = df.xs('Close', axis=1, level=1)

returns_df = df.pct_change().dropna()
market_returns = returns_df['^GSPC']

window_short = 50
window_medium = 147

print("="*80)
print("FEATURE ENGINEERING - MULTI-TIMEFRAME APPROACH")
print("="*80)
print(f"Short-term window: {window_short} hours (~1 week)")
print(f"Medium-term window: {window_medium} hours (~3 weeks)")

# SHORT-TERM FEATURES
rolling_vol_short = returns_df.rolling(window=window_short).std()

rolling_cov_mkt_short = returns_df.rolling(window=window_short).cov(market_returns)
rolling_mkt_var_short = market_returns.rolling(window=window_short).var()
rolling_beta_spx_short = rolling_cov_mkt_short.divide(rolling_mkt_var_short, axis=0)

non_spx_returns = returns_df.drop(columns=['^GSPC'], errors='ignore').dropna(axis=1, how='all')
sector_sum = non_spx_returns.sum(axis=1)
n_stocks = non_spx_returns.count(axis=1)

rolling_beta_sector_short = pd.DataFrame(index=returns_df.index, columns=non_spx_returns.columns)
for ticker in non_spx_returns.columns:
    loo_sector = (sector_sum - non_spx_returns[ticker].fillna(0)) / (n_stocks - 1).clip(lower=1)
    cov = returns_df[ticker].rolling(window=window_short).cov(loo_sector)
    var = loo_sector.rolling(window=window_short).var()
    rolling_beta_sector_short[ticker] = cov / var

# MEDIUM-TERM FEATURES
rolling_vol_medium = returns_df.rolling(window=window_medium).std()

rolling_cov_mkt_medium = returns_df.rolling(window=window_medium).cov(market_returns)
rolling_mkt_var_medium = market_returns.rolling(window=window_medium).var()
rolling_beta_spx_medium = rolling_cov_mkt_medium.divide(rolling_mkt_var_medium, axis=0)

rolling_beta_sector_medium = pd.DataFrame(index=returns_df.index, columns=non_spx_returns.columns)
for ticker in non_spx_returns.columns:
    loo_sector = (sector_sum - non_spx_returns[ticker].fillna(0)) / (n_stocks - 1).clip(lower=1)
    cov = returns_df[ticker].rolling(window=window_medium).cov(loo_sector)
    var = loo_sector.rolling(window=window_medium).var()
    rolling_beta_sector_medium[ticker] = cov / var

# INSTANTANEOUS FEATURES
def calculate_rsi(data, window=70):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

rsi_df = df.apply(calculate_rsi)
momentum_5h = df.pct_change(periods=5)

# REGIME CHANGE INDICATORS
vol_regime_shift = (rolling_vol_short - rolling_vol_medium) / rolling_vol_medium
beta_spx_regime_shift = rolling_beta_spx_short - rolling_beta_spx_medium
beta_sector_regime_shift = rolling_beta_sector_short - rolling_beta_sector_medium

# ASSEMBLE ts_df
ts_data_list = []
for ticker in stocks:
    if ticker == '^GSPC' or ticker not in df.columns:
        continue
    temp_df = pd.DataFrame({
        'Price': df[ticker],
        'Returns': returns_df[ticker],
        'Vol_Short': rolling_vol_short[ticker],
        'Beta_SPX_Short': rolling_beta_spx_short[ticker],
        'Beta_Sector_Short': rolling_beta_sector_short[ticker],
        'Vol_Medium': rolling_vol_medium[ticker],
        'Beta_SPX_Medium': rolling_beta_spx_medium[ticker],
        'Beta_Sector_Medium': rolling_beta_sector_medium[ticker],
        'RSI': rsi_df[ticker],
        'Momentum_5H': momentum_5h[ticker],
        'Vol_Regime_Shift': vol_regime_shift[ticker],
        'Beta_SPX_Regime_Shift': beta_spx_regime_shift[ticker],
        'Beta_Sector_Regime_Shift': beta_sector_regime_shift[ticker],
    }, index=df.index)
    temp_df['Ticker'] = ticker
    ts_data_list.append(temp_df)

if ts_data_list:
    ts_df = pd.concat(ts_data_list).reset_index().set_index(['Datetime', 'Ticker'])
    initial_rows = len(ts_df)
    ts_df = ts_df.dropna()
    dropped_rows = initial_rows - len(ts_df)

    print(f"\nTotal rows: {len(ts_df):,}")
    print(f"Rows dropped (NaN): {dropped_rows:,} ({dropped_rows/initial_rows:.1%})")
    print(f"Date range: {ts_df.index.get_level_values('Datetime').min()} to {ts_df.index.get_level_values('Datetime').max()}")
    print(f"Unique tickers: {ts_df.index.get_level_values('Ticker').nunique()}")
    print(f"Feature columns: {len([c for c in ts_df.columns if c not in ['Price', 'Returns']])}")

print("\nFEATURE ENGINEERING COMPLETE")

FEATURE ENGINEERING - MULTI-TIMEFRAME APPROACH
Short-term window: 50 hours (~1 week)
Medium-term window: 147 hours (~3 weeks)



Total rows: 63,152
Rows dropped (NaN): 6,688 (9.6%)
Date range: 2025-03-24 15:30:00+00:00 to 2026-02-23 20:30:00+00:00
Unique tickers: 40
Feature columns: 11

FEATURE ENGINEERING COMPLETE


# DBSCAN Temporal Clustering

In [4]:
ts_df = ts_df[~ts_df.index.duplicated(keep='first')]

density = ts_df.groupby(level='Datetime').size()
valid_timestamps = density[density >= 5].index

print(f"{'='*80}")
print(f"TRANSIENT REGIME DETECTION - DBSCAN CLUSTERING")
print(f"{'='*80}")
print(f"Data Density: {len(valid_timestamps)} valid hourly timestamps")
print(f"Date Range: {valid_timestamps.min()} to {valid_timestamps.max()}")

features_to_cluster = [
    'Returns', 'Vol_Short', 'Beta_SPX_Short', 'Beta_Sector_Short',
    'RSI', 'Momentum_5H', 'Vol_Regime_Shift',
    'Beta_SPX_Regime_Shift', 'Beta_Sector_Regime_Shift'
]
print(f"\nUsing features: {features_to_cluster}")

print(f"\n{'='*80}")
print("PHASE 1: Running DBSCAN Clustering (Hourly Snapshots)")
print(f"{'='*80}")

MIN_SAMPLES = 3

cluster_results = []
cluster_quality_log = []

for i, ts in enumerate(valid_timestamps):
    try:
        snapshot = ts_df.xs(ts, level='Datetime')[features_to_cluster].dropna()
        if len(snapshot) < 5:
            continue

        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(snapshot)

        pca = PCA(n_components=0.90)
        X_pca = pca.fit_transform(X_scaled)

        # Adaptive eps via k-distance heuristic
        k = MIN_SAMPLES
        nbrs = NearestNeighbors(n_neighbors=k + 1).fit(X_pca)
        distances, _ = nbrs.kneighbors(X_pca)
        k_distances = np.sort(distances[:, k])

        # Try percentiles [50, 60, 70, 80] and pick best
        best_eps = None
        best_score = -np.inf
        best_labels = None

        for pct in [50, 60, 70, 80]:
            eps_try = float(np.percentile(k_distances, pct))
            if eps_try <= 0:
                continue

            db = DBSCAN(eps=eps_try, min_samples=MIN_SAMPLES, metric='euclidean')
            lbls = db.fit_predict(X_pca)

            n_clust = len(set(lbls)) - (1 if -1 in lbls else 0)
            noise_frac = (lbls == -1).sum() / len(lbls)

            # Score: weight cluster count heavily, penalize extreme noise
            score = n_clust * 2.0
            if noise_frac > 0.70:
                score -= 5.0
            elif noise_frac < 0.10:
                score -= 2.0
            elif 0.15 <= noise_frac <= 0.50:
                score += 2.0

            if score > best_score:
                best_score = score
                best_eps = eps_try
                best_labels = lbls

        if best_labels is None:
            # Fallback: use 70th percentile
            eps_fallback = float(np.percentile(k_distances, 70))
            db = DBSCAN(eps=max(eps_fallback, 0.1), min_samples=MIN_SAMPLES, metric='euclidean')
            best_labels = db.fit_predict(X_pca)

        labels_final = best_labels
        unique_clusters = len(set(labels_final)) - (1 if -1 in labels_final else 0)
        noise_count = (labels_final == -1).sum()
        noise_pct = noise_count / len(labels_final)
        total_stocks = len(labels_final)

        quality_metrics = {
            'Datetime': ts,
            'Total_Stocks': total_stocks,
            'Unique_Clusters': unique_clusters,
            'Noise_Count': noise_count,
            'Noise_Pct': noise_pct,
            'PCA_Components': X_pca.shape[1],
            'Variance_Explained': pca.explained_variance_ratio_.sum(),
        }

        is_valid = True
        skip_reason = None

        if unique_clusters < 1:
            is_valid = False
            skip_reason = "No clusters found (all noise)"
        elif noise_pct > 0.75:
            is_valid = False
            skip_reason = f"Too noisy ({noise_pct:.1%} noise)"
        elif unique_clusters > total_stocks * 0.75:
            is_valid = False
            skip_reason = f"Over-fragmented ({unique_clusters} clusters for {total_stocks} stocks)"

        quality_metrics['Is_Valid'] = is_valid
        quality_metrics['Skip_Reason'] = skip_reason
        cluster_quality_log.append(quality_metrics)

        if is_valid:
            snapshot['Cluster_ID'] = labels_final
            snapshot['Datetime'] = ts
            snapshot['Num_Clusters'] = unique_clusters
            snapshot['Noise_Pct'] = noise_pct
            cluster_results.append(snapshot.reset_index())

        if (i + 1) % 100 == 0:
            valid_so_far = len(cluster_results)
            print(f"  Processed {i+1}/{len(valid_timestamps)} timestamps... ({valid_so_far} valid so far)")

    except Exception as e:
        cluster_quality_log.append({
            'Datetime': ts, 'Total_Stocks': np.nan, 'Unique_Clusters': np.nan,
            'Noise_Count': np.nan, 'Noise_Pct': np.nan,
            'PCA_Components': np.nan, 'Variance_Explained': np.nan,
            'Is_Valid': False, 'Skip_Reason': f"Error: {str(e)[:50]}"
        })
        continue

if not cluster_results:
    raise ValueError("No valid clusters found.")

cluster_history = pd.concat(cluster_results, ignore_index=True)

print(f"\n{'='*80}")
print("PHASE 1 COMPLETE: Cluster Quality Summary")
print(f"{'='*80}")

df_quality = pd.DataFrame(cluster_quality_log)
total_timestamps = len(df_quality)
valid_timestamps_count = df_quality['Is_Valid'].sum()
invalid_timestamps_count = total_timestamps - valid_timestamps_count

print(f"\nTimestamp Analysis:")
print(f"  Total timestamps processed: {total_timestamps}")
print(f"  Valid clustering windows: {valid_timestamps_count} ({valid_timestamps_count/total_timestamps:.1%})")
print(f"  Invalid/skipped windows: {invalid_timestamps_count} ({invalid_timestamps_count/total_timestamps:.1%})")

if invalid_timestamps_count > 0:
    print(f"\nSkip Reasons:")
    skip_summary = df_quality[~df_quality['Is_Valid']]['Skip_Reason'].value_counts()
    for reason, count in skip_summary.items():
        print(f"  - {reason}: {count}")

valid_quality = df_quality[df_quality['Is_Valid']]
if len(valid_quality) > 0:
    print(f"\nValid Cluster Statistics:")
    print(f"  Avg clusters per timestamp: {valid_quality['Unique_Clusters'].mean():.1f}")
    print(f"  Avg noise percentage: {valid_quality['Noise_Pct'].mean():.1%}")
    print(f"  Avg PCA variance retained: {valid_quality['Variance_Explained'].mean():.1%}")

print(f"\nCluster History Generated:")
print(f"  Total rows: {len(cluster_history)}")
print(f"  Unique timestamps: {cluster_history['Datetime'].nunique()}")

TRANSIENT REGIME DETECTION - DBSCAN CLUSTERING
Data Density: 1579 valid hourly timestamps
Date Range: 2025-03-24 15:30:00+00:00 to 2026-02-23 20:30:00+00:00

Using features: ['Returns', 'Vol_Short', 'Beta_SPX_Short', 'Beta_Sector_Short', 'RSI', 'Momentum_5H', 'Vol_Regime_Shift', 'Beta_SPX_Regime_Shift', 'Beta_Sector_Regime_Shift']

PHASE 1: Running DBSCAN Clustering (Hourly Snapshots)


  Processed 100/1579 timestamps... (100 valid so far)


  Processed 200/1579 timestamps... (200 valid so far)


  Processed 300/1579 timestamps... (300 valid so far)


  Processed 400/1579 timestamps... (400 valid so far)


...

  Processed 1200/1579 timestamps... (1200 valid so far)


  Processed 1300/1579 timestamps... (1300 valid so far)


  Processed 1400/1579 timestamps... (1400 valid so far)


  Processed 1500/1579 timestamps... (1500 valid so far)



PHASE 1 COMPLETE: Cluster Quality Summary

Timestamp Analysis:
  Total timestamps processed: 1579
  Valid clustering windows: 1579 (100.0%)
  Invalid/skipped windows: 0 (0.0%)

Valid Cluster Statistics:
  Avg clusters per timestamp: 1.5
  Avg noise percentage: 28.0%
  Avg PCA variance retained: 92.9%

Cluster History Generated:
  Total rows: 63152
  Unique timestamps: 1579


# Pair Stability Analysis

In [5]:
print(f"\n{'='*80}")
print("PHASE 2: Analyzing Cluster Stability")
print(f"{'='*80}")

pair_co_cluster_freq = {}

for ts in cluster_history['Datetime'].unique():
    snapshot = cluster_history[cluster_history['Datetime'] == ts]
    for cluster_id in snapshot['Cluster_ID'].unique():
        if cluster_id == -1:
            continue
        members = snapshot[snapshot['Cluster_ID'] == cluster_id]['Ticker'].tolist()
        for s1, s2 in itertools.combinations(sorted(members), 2):
            pair_key = (s1, s2)
            if pair_key not in pair_co_cluster_freq:
                pair_co_cluster_freq[pair_key] = 0
            pair_co_cluster_freq[pair_key] += 1

total_valid_windows = cluster_history['Datetime'].nunique()

pair_stability_data = []
for pair, count in pair_co_cluster_freq.items():
    frequency = count / total_valid_windows
    pair_stability_data.append({
        'Ticker_1': pair[0],
        'Ticker_2': pair[1],
        'Pair': f"{pair[0]}-{pair[1]}",
        'Co_Cluster_Count': count,
        'Co_Cluster_Frequency': frequency,
        'Is_Stable': frequency > 0.25
    })

df_pair_stability = pd.DataFrame(pair_stability_data).sort_values('Co_Cluster_Frequency', ascending=False)

print(f"\nPair Clustering Analysis:")
print(f"  Total unique pairs observed: {len(df_pair_stability)}")
print(f"  Pairs clustered together >50% of time: {(df_pair_stability['Co_Cluster_Frequency'] > 0.50).sum()}")
print(f"  Pairs clustered together >30% of time: {(df_pair_stability['Co_Cluster_Frequency'] > 0.30).sum()}")
print(f"  Pairs clustered together <10% of time: {(df_pair_stability['Co_Cluster_Frequency'] < 0.10).sum()}")

print(f"\n{'='*80}")
print("TOP 15 MOST FREQUENTLY CO-CLUSTERED PAIRS")
print(f"{'='*80}")
print(df_pair_stability[['Pair', 'Co_Cluster_Count', 'Co_Cluster_Frequency']].head(15).to_string(index=False))

print(f"\n{'='*80}")
print("TOP 15 MOST TRANSIENT PAIRS (Rare Co-Clustering)")
print(f"{'='*80}")
transient_pairs = df_pair_stability[
    (df_pair_stability['Co_Cluster_Frequency'] > 0.05) &
    (df_pair_stability['Co_Cluster_Frequency'] < 0.20)
].sort_values('Co_Cluster_Frequency', ascending=True)
print(transient_pairs[['Pair', 'Co_Cluster_Count', 'Co_Cluster_Frequency']].head(15).to_string(index=False))


PHASE 2: Analyzing Cluster Stability



Pair Clustering Analysis:
  Total unique pairs observed: 780
  Pairs clustered together >50% of time: 333
  Pairs clustered together >30% of time: 637
  Pairs clustered together <10% of time: 77

TOP 15 MOST FREQUENTLY CO-CLUSTERED PAIRS
     Pair  Co_Cluster_Count  Co_Cluster_Frequency
 ADI-NXPI              1417              0.897403
 NXPI-TSM              1358              0.860038
LRCX-NXPI              1347              0.853072
 LRCX-TSM              1341              0.849272
  ADI-TSM              1340              0.848638
NXPI-SWKS              1336              0.846105
KLAC-LRCX              1336              0.846105
AMAT-LRCX              1328              0.841039
 ADI-LRCX              1324              0.838505
 ADI-SWKS              1307              0.827739
QRVO-SWKS              1305              0.826472
 ADI-AMAT              1293              0.818873
  ADI-TXN              1287              0.815073
AMAT-NXPI              1283              0.812540
KLAC-NXPI  

# Formation & Dissolution Detection

In [6]:
print(f"{'='*80}")
print("PHASE 2B: CLUSTER FORMATION & DISSOLUTION EVENTS")
print(f"{'='*80}")

all_pairs = list(pair_co_cluster_freq.keys())
all_timestamps = sorted(cluster_history['Datetime'].unique())

print(f"Tracking {len(all_pairs)} pairs across {len(all_timestamps)} timestamps...")

pair_coclustering = {}
for ts in all_timestamps:
    snapshot = cluster_history[cluster_history['Datetime'] == ts]
    coclustered_at_ts = set()
    for cluster_id in snapshot['Cluster_ID'].unique():
        if cluster_id == -1:
            continue
        members = sorted(snapshot[snapshot['Cluster_ID'] == cluster_id]['Ticker'].tolist())
        for s1, s2 in itertools.combinations(members, 2):
            coclustered_at_ts.add((s1, s2))
    for pair in all_pairs:
        if pair not in pair_coclustering:
            pair_coclustering[pair] = []
        pair_coclustering[pair].append(1 if pair in coclustered_at_ts else 0)

MIN_GAP_HOURS = 5
MIN_EPISODE_HOURS = 5

formation_events = []
dissolution_events = []
cluster_durations = []

for pair, series in pair_coclustering.items():
    ts_series = list(zip(all_timestamps, series))
    in_cluster = False
    formation_ts = None
    formation_idx = None
    gap_count = MIN_GAP_HOURS

    for i, (ts, val) in enumerate(ts_series):
        if val == 1:
            if not in_cluster and gap_count >= MIN_GAP_HOURS:
                in_cluster = True
                formation_ts = ts
                formation_idx = i
                formation_events.append({
                    'Ticker_1': pair[0], 'Ticker_2': pair[1],
                    'Pair': f"{pair[0]}-{pair[1]}",
                    'Formation_Time': ts, 'Timestamp_Index': i
                })
            elif not in_cluster:
                in_cluster = True
                formation_ts = ts
                formation_idx = i
            gap_count = 0
        else:
            if in_cluster:
                gap_count += 1
                if gap_count >= MIN_GAP_HOURS:
                    last_cocluster_idx = i - gap_count
                    dissolution_ts = all_timestamps[min(last_cocluster_idx + 1, len(all_timestamps) - 1)]
                    duration = max(1, last_cocluster_idx - formation_idx + 1) if formation_idx is not None else 1
                    dissolution_events.append({
                        'Ticker_1': pair[0], 'Ticker_2': pair[1],
                        'Pair': f"{pair[0]}-{pair[1]}",
                        'Dissolution_Time': dissolution_ts, 'Duration_Hours': duration
                    })
                    if formation_ts is not None:
                        cluster_durations.append({
                            'Ticker_1': pair[0], 'Ticker_2': pair[1],
                            'Pair': f"{pair[0]}-{pair[1]}",
                            'Formation_Time': formation_ts,
                            'Dissolution_Time': dissolution_ts,
                            'Duration_Hours': duration
                        })
                    in_cluster = False
                    formation_ts = None
                    formation_idx = None
            else:
                gap_count += 1

    if in_cluster and formation_ts is not None:
        last_cocluster_idx = len(ts_series) - 1
        for j in range(len(ts_series) - 1, -1, -1):
            if ts_series[j][1] == 1:
                last_cocluster_idx = j
                break
        duration = max(1, last_cocluster_idx - formation_idx + 1) if formation_idx is not None else 1
        dissolution_ts = all_timestamps[min(last_cocluster_idx + 1, len(all_timestamps) - 1)]
        dissolution_events.append({
            'Ticker_1': pair[0], 'Ticker_2': pair[1],
            'Pair': f"{pair[0]}-{pair[1]}",
            'Dissolution_Time': dissolution_ts, 'Duration_Hours': duration
        })
        cluster_durations.append({
            'Ticker_1': pair[0], 'Ticker_2': pair[1],
            'Pair': f"{pair[0]}-{pair[1]}",
            'Formation_Time': formation_ts,
            'Dissolution_Time': dissolution_ts,
            'Duration_Hours': duration
        })

df_formations = pd.DataFrame(formation_events)
df_dissolutions = pd.DataFrame(dissolution_events)
df_durations = pd.DataFrame(cluster_durations)

print(f"\nFormation events detected: {len(df_formations)}")
print(f"Dissolution events detected: {len(df_dissolutions)}")
print(f"Complete cluster episodes: {len(df_durations)}")

if len(df_durations) > 0:
    print(f"\nDuration Statistics (hours):")
    print(f"  Mean:   {df_durations['Duration_Hours'].mean():.1f}")
    print(f"  Median: {df_durations['Duration_Hours'].median():.1f}")
    print(f"  Min:    {df_durations['Duration_Hours'].min():.0f}")
    print(f"  Max:    {df_durations['Duration_Hours'].max():.0f}")

# Pair classification
print(f"\n{'='*80}")
print("PAIR CLASSIFICATION")
print(f"{'='*80}")

pair_formation_counts = df_formations.groupby('Pair').size().reset_index(name='Formation_Count')
pair_avg_duration = df_durations.groupby('Pair')['Duration_Hours'].mean().reset_index(name='Avg_Duration')

pair_classification = pair_formation_counts.merge(pair_avg_duration, on='Pair', how='left')
pair_classification = pair_classification.merge(
    df_pair_stability[['Pair', 'Co_Cluster_Frequency']], on='Pair', how='left'
)

pair_classification['Category'] = 'unknown'
transient_mask = (pair_classification['Formation_Count'] >= 3) & (pair_classification['Avg_Duration'] <= 30)
pair_classification.loc[transient_mask, 'Category'] = 'transient'

stable_mask = ((pair_classification['Co_Cluster_Frequency'] > 0.25) | (pair_classification['Avg_Duration'] > 100)) & (pair_classification['Category'] == 'unknown')
pair_classification.loc[stable_mask, 'Category'] = 'stable_candidate'

sporadic_mask = (pair_classification['Formation_Count'] <= 2) & (pair_classification['Category'] == 'unknown')
pair_classification.loc[sporadic_mask, 'Category'] = 'sporadic'

for cat in ['transient', 'stable_candidate', 'sporadic', 'unknown']:
    count = len(pair_classification[pair_classification['Category'] == cat])
    print(f"  {cat:20s}: {count} pairs")

# Actionable formations
if len(df_durations) > 0:
    df_formations_actionable = df_formations.merge(
        df_durations[['Pair', 'Formation_Time', 'Duration_Hours']],
        on=['Pair', 'Formation_Time'], how='inner'
    ).query('Duration_Hours >= @MIN_EPISODE_HOURS')
    print(f"\nActionable formations (duration >= {MIN_EPISODE_HOURS}h): {len(df_formations_actionable)}")
else:
    df_formations_actionable = df_formations.copy()

print(f"\nFORMATION/DISSOLUTION ANALYSIS COMPLETE")

PHASE 2B: CLUSTER FORMATION & DISSOLUTION EVENTS
Tracking 780 pairs across 1579 timestamps...



Formation events detected: 27663
Dissolution events detected: 27663
Complete cluster episodes: 27663

Duration Statistics (hours):
  Mean:   26.4
  Median: 9.0
  Min:    1
  Max:    963

PAIR CLASSIFICATION
  transient           : 474 pairs
  stable_candidate    : 306 pairs
  sporadic            : 0 pairs
  unknown             : 0 pairs

Actionable formations (duration >= 5h): 17507

FORMATION/DISSOLUTION ANALYSIS COMPLETE


# Correctness Checks

In [7]:
print(f"{'='*80}")
print("CORRECTNESS CHECK 1: Feature-Shuffle Permutation Test (DBSCAN)")
print(f"{'='*80}")

n_permutations = 30
n_sample_timestamps = 80

all_ts_index = ts_df.index.get_level_values('Datetime').unique()
rng = np.random.default_rng(42)
if len(all_ts_index) > n_sample_timestamps:
    sample_ts = rng.choice(all_ts_index, size=n_sample_timestamps, replace=False)
else:
    sample_ts = all_ts_index

scale = len(sample_ts) / total_valid_windows
null_counts = {}

for perm in range(n_permutations):
    perm_counts = {}
    for ts in sample_ts:
        try:
            snapshot = ts_df.xs(ts, level='Datetime')[features_to_cluster].dropna()
        except KeyError:
            continue
        if len(snapshot) < 5:
            continue

        tickers = snapshot.index.tolist()
        values = snapshot.values.copy()
        np.random.shuffle(values)

        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(values)
        pca_perm = PCA(n_components=0.90)
        X_pca = pca_perm.fit_transform(X_scaled)

        # Adaptive eps on shuffled data
        k = MIN_SAMPLES
        nbrs = NearestNeighbors(n_neighbors=k + 1).fit(X_pca)
        distances, _ = nbrs.kneighbors(X_pca)
        k_distances_perm = np.sort(distances[:, k])

        best_eps_perm = None
        best_score_perm = -np.inf
        best_labels_perm = None

        for pct in [50, 60, 70, 80]:
            eps_try = float(np.percentile(k_distances_perm, pct))
            if eps_try <= 0:
                continue
            db = DBSCAN(eps=eps_try, min_samples=MIN_SAMPLES, metric='euclidean')
            lbls = db.fit_predict(X_pca)
            n_clust = len(set(lbls)) - (1 if -1 in lbls else 0)
            noise_frac = (lbls == -1).sum() / len(lbls)

            # Score: weight cluster count heavily, penalize extreme noise
            score = n_clust * 2.0
            if noise_frac > 0.70:
                score -= 5.0
            elif noise_frac < 0.10:
                score -= 2.0
            elif 0.15 <= noise_frac <= 0.50:
                score += 2.0

            if score > best_score_perm:
                best_score_perm = score
                best_labels_perm = lbls

        if best_labels_perm is None:
            eps_fallback = float(np.percentile(k_distances_perm, 70))
            db = DBSCAN(eps=max(eps_fallback, 0.1), min_samples=MIN_SAMPLES)
            best_labels_perm = db.fit_predict(X_pca)

        perm_labels = best_labels_perm

        for cid in set(perm_labels):
            if cid == -1:
                continue
            members = sorted([tickers[j] for j in range(len(tickers)) if perm_labels[j] == cid])
            for s1, s2 in itertools.combinations(members, 2):
                key = (s1, s2)
                perm_counts[key] = perm_counts.get(key, 0) + 1

    for key, cnt in perm_counts.items():
        null_counts.setdefault(key, []).append(cnt)

    if (perm + 1) % 10 == 0:
        print(f"  Permutation {perm+1}/{n_permutations} complete")

for key in null_counts:
    while len(null_counts[key]) < n_permutations:
        null_counts[key].append(0)

pair_zscores = {}
for pair, obs_count in pair_co_cluster_freq.items():
    obs_scaled = obs_count * scale
    null_vals = np.array(null_counts.get(pair, [0] * n_permutations), dtype=float)
    null_mean = null_vals.mean()
    null_std = null_vals.std()
    if null_std > 0:
        z = (obs_scaled - null_mean) / null_std
    else:
        z = 0.0 if obs_scaled <= null_mean else np.inf
    pair_zscores[pair] = float(z)

n_significant = sum(1 for z in pair_zscores.values() if z > 1.96)
frac_sig = n_significant / len(pair_zscores) if pair_zscores else 0.0

print(f"\nFraction of pairs significant at p<0.05: {frac_sig:.1%}")

top_z = sorted(pair_zscores.items(), key=lambda x: x[1], reverse=True)[:15]
print(f"\nTop 15 pairs by permutation Z-score:")
for pair, z in top_z:
    print(f"  {pair[0]:6s}-{pair[1]:6s}  Z = {z:.2f}")

# OOS split
print(f"\n{'='*80}")
print("CORRECTNESS CHECK 2: Out-of-Sample Split")
print(f"{'='*80}")

all_dates = sorted(cluster_history['Datetime'].unique())
split_point = int(len(all_dates) * 0.67)
split_timestamp = all_dates[split_point]

train_history = cluster_history[cluster_history['Datetime'] <= split_timestamp]
test_history = cluster_history[cluster_history['Datetime'] > split_timestamp]

print(f"\nSplit point: {split_timestamp}")
print(f"Training period: {all_dates[0]} to {split_timestamp} ({len(train_history)} rows)")
print(f"Test period: {split_timestamp} to {all_dates[-1]} ({len(test_history)} rows)")

def calc_pair_freq(history_df):
    freq = {}
    total = history_df['Datetime'].nunique()
    for ts in history_df['Datetime'].unique():
        snap = history_df[history_df['Datetime'] == ts]
        for cid in snap['Cluster_ID'].unique():
            if cid == -1:
                continue
            members = sorted(snap[snap['Cluster_ID'] == cid]['Ticker'].tolist())
            for s1, s2 in itertools.combinations(members, 2):
                freq[(s1, s2)] = freq.get((s1, s2), 0) + 1
    return {k: v / total for k, v in freq.items()}

train_freq = calc_pair_freq(train_history)
test_freq = calc_pair_freq(test_history)

common_pairs = set(train_freq.keys()) & set(test_freq.keys())
if len(common_pairs) > 0:
    train_vals = [train_freq[p] for p in common_pairs]
    test_vals = [test_freq[p] for p in common_pairs]
    from scipy.stats import pearsonr
    oos_corr, oos_pval = pearsonr(train_vals, test_vals)
    print(f"\nPairs observed in both periods: {len(common_pairs)}")
    print(f"Correlation of co-clustering frequency (train vs test): {oos_corr:.3f} (p={oos_pval:.4f})")
    if oos_corr > 0.5:
        print("RESULT: Good out-of-sample stability")
    elif oos_corr > 0.2:
        print("RESULT: Moderate out-of-sample stability")
    else:
        print("WARNING: Poor out-of-sample stability")

oos_split_timestamp = split_timestamp

print(f"\n{'='*80}")
print("ALL CORRECTNESS CHECKS COMPLETE")
print(f"{'='*80}")

CORRECTNESS CHECK 1: Feature-Shuffle Permutation Test (DBSCAN)


  Permutation 10/30 complete


  Permutation 20/30 complete


  Permutation 30/30 complete

Fraction of pairs significant at p<0.05: 29.7%

Top 15 pairs by permutation Z-score:
  NXPI  -SWKS    Z = 9.60
  ADI   -AMAT    Z = 8.94
  ADI   -NXPI    Z = 8.53
  ADI   -TSM     Z = 8.41
  KLAC  -NXPI    Z = 8.13
  LRCX  -NXPI    Z = 8.11
  NXPI  -STM     Z = 7.92
  NXPI  -QRVO    Z = 7.90
  ADI   -SWKS    Z = 7.83
  NXPI  -TSM     Z = 7.14
  NXPI  -TXN     Z = 7.03
  AMAT  -TSM     Z = 7.01
  AMAT  -KLAC    Z = 6.90
  ADI   -ASML    Z = 6.60
  AMAT  -NXPI    Z = 6.59

CORRECTNESS CHECK 2: Out-of-Sample Split

Split point: 2025-10-30 16:30:00+00:00
Training period: 2025-03-24 15:30:00+00:00 to 2025-10-30 16:30:00+00:00 (42314 rows)
Test period: 2025-10-30 16:30:00+00:00 to 2026-02-23 20:30:00+00:00 (20838 rows)



Pairs observed in both periods: 780
Correlation of co-clustering frequency (train vs test): 0.839 (p=0.0000)
RESULT: Good out-of-sample stability

ALL CORRECTNESS CHECKS COMPLETE


# Save Artifacts

In [8]:
import os
import pickle

data_dir = os.path.join(os.path.dirname(os.path.abspath('__file__')), 'data')
os.makedirs(data_dir, exist_ok=True)

save_items = {
    'dbscan_ts_df': ts_df,
    'dbscan_df_formations': df_formations,
    'dbscan_df_formations_actionable': df_formations_actionable,
    'dbscan_pair_classification': pair_classification,
    'dbscan_cluster_history': cluster_history,
    'dbscan_df_durations': df_durations,
    'dbscan_df_pair_stability': df_pair_stability,
    'dbscan_oos_split_timestamp': oos_split_timestamp,
    'dbscan_pair_co_cluster_freq': pair_co_cluster_freq,
}

for name, obj in save_items.items():
    path = os.path.join(data_dir, f'{name}.pkl')
    with open(path, 'wb') as f:
        pickle.dump(obj, f)
    size = len(obj) if hasattr(obj, '__len__') else 1
    print(f'Saved {name} -> {path} ({size} items)')

print(f'\nAll DBSCAN artifacts saved to {data_dir}/')

Saved dbscan_ts_df -> /Users/jack/Documents/code/pairs-test/research/data/dbscan_ts_df.pkl (63152 items)
Saved dbscan_df_formations -> /Users/jack/Documents/code/pairs-test/research/data/dbscan_df_formations.pkl (27663 items)
Saved dbscan_df_formations_actionable -> /Users/jack/Documents/code/pairs-test/research/data/dbscan_df_formations_actionable.pkl (17507 items)
Saved dbscan_pair_classification -> /Users/jack/Documents/code/pairs-test/research/data/dbscan_pair_classification.pkl (780 items)
Saved dbscan_cluster_history -> /Users/jack/Documents/code/pairs-test/research/data/dbscan_cluster_history.pkl (63152 items)
Saved dbscan_df_durations -> /Users/jack/Documents/code/pairs-test/research/data/dbscan_df_durations.pkl (27663 items)
Saved dbscan_df_pair_stability -> /Users/jack/Documents/code/pairs-test/research/data/dbscan_df_pair_stability.pkl (780 items)
Saved dbscan_oos_split_timestamp -> /Users/jack/Documents/code/pairs-test/research/data/dbscan_oos_split_timestamp.pkl (1 items)
